In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style

%matplotlib inline
import plotly
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

### Reading data from Johnhopkins Github repository

In [2]:
Confirmed_filepath = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
Death_filepath = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
Recovered_filepath = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

In [3]:
 def Import_COVIDData(filepath):
        data = filepath
        data = pd.read_csv(data)
        data.rename(columns={'Province/State':'Province','Country/Region':'Country'},inplace = True)
        return data      

In [4]:
def transform_COVIDData(data):
        data = pd.melt(
        frame = data,
        id_vars = ['Province','Country','Lat','Long','Status'],
        var_name = 'Date',
        value_name = 'Count'
    )
        return data

#### Confirmed Cases across globe

In [5]:
ConfirmedCases = Import_COVIDData(Confirmed_filepath)
ConfirmedCases['Status'] = 'Confirmed'
ConfirmedCases = transform_COVIDData(ConfirmedCases)

#### Death Cases across globe

In [6]:
DeathCases = Import_COVIDData(Death_filepath)
DeathCases['Status'] = 'Dealth'
DeathCases = transform_COVIDData(DeathCases)


#### Recovered Cases across globe

In [7]:
RecoveredCases = Import_COVIDData(Recovered_filepath)
RecoveredCases['Status'] = 'Recovered'
RecoveredCases = transform_COVIDData(RecoveredCases)


### Merge data

In [8]:
Totaldata = pd.concat([ConfirmedCases,DeathCases,RecoveredCases])

In [9]:
Totaldata.dtypes

Province     object
Country      object
Lat         float64
Long        float64
Status       object
Date         object
Count         int64
dtype: object

In [10]:
Totaldata['Date'] = pd.to_datetime(Totaldata['Date'])
Totaldata.set_index('Date',inplace= True)

In [39]:
Totaldata

,Province,Country,Lat,Long,Status,Count
Date,,,,,,
2020-01-22,NaN,Afghanistan,33.939110,67.709953,Confirmed,0
2020-01-22,NaN,Albania,41.153300,20.168300,Confirmed,0
2020-01-22,NaN,Algeria,28.033900,1.659600,Confirmed,0
2020-01-22,NaN,Andorra,42.506300,1.521800,Confirmed,0
2020-01-22,NaN,Angola,-11.202700,17.873900,Confirmed,0
...,...,...,...,...,...,...
2021-05-10,NaN,Vietnam,14.058324,108.277199,Recovered,2618
2021-05-10,NaN,West Bank and Gaza,31.952200,35.233200,Recovered,287206
2021-05-10,NaN,Yemen,15.552727,48.516388,Recovered,3001


In [152]:
date =Totaldata.index[-1]
TotaldataConfirmed = Totaldata[(Totaldata['Status'] == 'Confirmed') & (Totaldata.index == date)]
TotaldataRecovered = Totaldata[(Totaldata['Status'] == 'Recovered') & (Totaldata.index == date)]
TotaldataDealth = Totaldata[(Totaldata['Status'] == 'Dealth') & (Totaldata.index == date)]

In [153]:
dt = list(set(Totaldata.index))
dt.sort()
date =dt[-2]
TotaldataConfirmed_1dayMinus = Totaldata[(Totaldata['Status'] == 'Confirmed') & (Totaldata.index == date)]
TotaldataRecovered_1dayMinus = Totaldata[(Totaldata['Status'] == 'Recovered') & (Totaldata.index == date)]
TotaldataDealth_1dayMinus = Totaldata[(Totaldata['Status'] == 'Dealth') & (Totaldata.index == date)]

In [63]:
TotaldataConfirmed['Count'].sum()
TotaldataRecovered['Count'].sum()
TotaldataDealth['Count'].sum()

3303696

In [157]:
fig = go.Figure()
fig.add_trace(go.Indicator(
    mode = "number+delta",
    title = {'text': "Confirmed"},
    value = TotaldataConfirmed['Count'].sum(),
    delta = {'reference': TotaldataConfirmed_1dayMinus['Count'].sum()},
    domain = {'row': 0, 'column': 0}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    title = {'text': "Recovered"},
    value = TotaldataRecovered['Count'].sum(),
    delta = {'reference': TotaldataRecovered_1dayMinus['Count'].sum()},
    domain = {'row': 0, 'column': 1}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    title = {'text': "Dealth"},
    number = {'valueformat':'0.2s'},
    value = TotaldataDealth['Count'].sum(),
    delta = {'reference': TotaldataDealth_1dayMinus['Count'].sum()},
    domain = {'row': 1, 'column': 0}))

fig.update_layout(
    grid = {'rows': 2, 'columns': 2, 'pattern': "independent"},
    template = {'data' : {'indicator': [{
        'title': {'text': "Speed"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': 90}}]
                         }})

### Top 10 Countries with recovered cases on the most recent days

In [11]:
def RecentCases_Top10Countries(Status):
    Top10Countries = Totaldata[(Totaldata['Status']==Status) & (Totaldata.index==max(Totaldata[Totaldata['Status']==Status].index))].sort_values(by='Count',ascending = False).head(10)[['Country','Count']]
    return Top10Countries

In [12]:
Top10CountriesRecovered = RecentCases_Top10Countries('Recovered')
Top10CountriesConfirmed = RecentCases_Top10Countries('Confirmed')
Top10CountriesDealth = RecentCases_Top10Countries('Dealth')

In [13]:
def Display_Top10Country(Top10Countries,Status):
    Top10Countries = Top10Countries.reset_index()
    Top10Countries = Top10Countries[['Country','Count']]
    date =max(Totaldata[Totaldata['Status']==Status].index)
    date = date.date()
    print(Status +' cases till:',date)
    return Top10Countries

In [14]:
Top10CountriesRecovered = Display_Top10Country(Top10CountriesRecovered,'Recovered')
Top10CountriesRecovered.style.background_gradient(cmap = 'Reds')

Recovered cases till: 2021-05-10


,Country,Count
0,India,19027304
1,Brazil,13481062
2,Turkey,4743871
3,Russia,4451443
4,Italy,3619586
5,Germany,3188710
6,Argentina,2837058
7,Colombia,2835554
8,Poland,2576034
9,Iran,2127192


In [15]:
Top10CountriesConfirmed = Display_Top10Country(Top10CountriesConfirmed,'Confirmed')
Top10CountriesConfirmed.style.background_gradient(cmap = 'Reds')

Confirmed cases till: 2021-05-10


,Country,Count
0,US,32743981
1,India,22992517
2,Brazil,15209990
3,France,5730585
4,Turkey,5044936
5,Russia,4832959
6,United Kingdom,4437217
7,Italy,4116287
8,Spain,3581392
9,Germany,3538208


In [16]:
Top10CountriesDealth = Display_Top10Country(Top10CountriesConfirmed,'Dealth')
Top10CountriesDealth.style.background_gradient(cmap = 'Reds')

Dealth cases till: 2021-05-10


,Country,Count
0,US,32743981
1,India,22992517
2,Brazil,15209990
3,France,5730585
4,Turkey,5044936
5,Russia,4832959
6,United Kingdom,4437217
7,Italy,4116287
8,Spain,3581392
9,Germany,3538208


In [52]:
date =max(Totaldata[Totaldata['Status']=='Confirmed'].index)
date = date.date()
px.bar(Top10CountriesRecovered,x='Country',y='Count',color='Country',title='Top 10 Recovered cases till '+str(date))


In [62]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Bar(x=Top10CountriesRecovered['Country'],y=Top10CountriesRecovered['Count'],text = Top10CountriesRecovered['Count'],name ='Recovered'),
    row=1, col=1
)


fig.add_trace(
    go.Bar(x=Top10CountriesConfirmed['Country'],y=Top10CountriesConfirmed['Count'],text = Top10CountriesConfirmed['Count'],name ='Confirmed'),
    row=2, col=1
)

fig.add_trace(   
    go.Bar(x=Top10CountriesDealth['Country'],y=Top10CountriesDealth['Count'],text = Top10CountriesDealth['Count'],name ='Dealth'),
    row=3, col=1
)


fig.update_layout(height=1400, width=800, title_text="Top 10 Countries as of "+str(date))
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()
